# Surface Circulation — NEMO GYRE

Analyse surface currents from the 6-month GYRE simulation.
- Surface current vectors
- Surface kinetic energy

In [ ]:
from pathlib import Path

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

plt.rcParams.update({"font.size": 16, "axes.titlesize": 18, "axes.labelsize": 14})

OUTPUT_DIR = Path("../output")

## Load simulation output

Load grid_U, grid_V (velocities) and grid_T (coordinates). Velocities
live on staggered grids (U- and V-points); for visualisation they are
plotted on the T-grid coordinates without interpolation.

In [ ]:
ds_u = xr.open_mfdataset(sorted(OUTPUT_DIR.glob("*_grid_U.nc")))
ds_v = xr.open_mfdataset(sorted(OUTPUT_DIR.glob("*_grid_V.nc")))
ds_t = xr.open_mfdataset(sorted(OUTPUT_DIR.glob("*_grid_T.nc")))

mask_ds = xr.open_dataset(OUTPUT_DIR / "mesh_mask.nc", decode_times=False)

## Grid geometry and masking

The GYRE grid's i-axis is rotated 45° from geographic east. Velocity
components must be rotated from grid-aligned (i, j) to geographic
(east, north) before plotting. The interior mask excludes a 2-cell
border to avoid staggered-grid boundary artefacts.

In [ ]:
# Grid angle: direction of i-axis relative to geographic east
glamt = mask_ds.glamt.isel(time_counter=0)
gphit = mask_ds.gphit.isel(time_counter=0)
dx_lon = glamt.diff("x").pad(x=(0, 1), mode="edge")
dx_lat = gphit.diff("x").pad(x=(0, 1), mode="edge")
grid_angle = np.arctan2(dx_lat, dx_lon)


def rotate_to_geo(ui, vj, angle):
    """Rotate (i,j)-aligned vectors to geographic (east, north)."""
    u_east = ui * np.cos(angle) - vj * np.sin(angle)
    v_north = ui * np.sin(angle) + vj * np.cos(angle)
    return u_east, v_north


# Interior mask — 2-cell border for velocity fields
tmask_sfc = mask_ds.tmask.isel(time_counter=0, nav_lev=0)
ny, nx = tmask_sfc.sizes["y"], tmask_sfc.sizes["x"]
not_boundary = (
    (xr.DataArray(range(ny), dims="y") > 1)
    & (xr.DataArray(range(ny), dims="y") < ny - 2)
    & (xr.DataArray(range(nx), dims="x") > 1)
    & (xr.DataArray(range(nx), dims="x") < nx - 2)
)
interior = tmask_sfc.where(not_boundary, 0)

# Map projection
MARGIN = 0.5
proj = ccrs.Stereographic(central_longitude=-68, central_latitude=32)
extent = [
    float(ds_t.nav_lon.min()) - MARGIN, float(ds_t.nav_lon.max()) + MARGIN,
    float(ds_t.nav_lat.min()) - MARGIN, float(ds_t.nav_lat.max()) + MARGIN,
]

## Surface current vectors

Time-mean surface velocity field. The quiver plot should show the
double-gyre pattern: subtropical anticyclonic gyre in the south,
subpolar cyclonic gyre in the north, with a strong western boundary
current.

In [ ]:
# Time-mean surface velocities, rotated to geographic
u_sfc = ds_u["vozocrtx"].isel(depthu=0).mean("time_counter")
v_sfc = ds_v["vomecrty"].isel(depthv=0).mean("time_counter")
u_east, v_north = rotate_to_geo(u_sfc, v_sfc, grid_angle)
speed = np.sqrt(u_east**2 + v_north**2).where(interior)

fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection=proj))
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor="tan", edgecolor="k", linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.gridlines(draw_labels=False, alpha=0.3)

pcm = ax.pcolormesh(
    ds_t.nav_lon.values, ds_t.nav_lat.values, speed.values,
    shading="auto", cmap="cividis", alpha=0.7, transform=ccrs.PlateCarree(),
)
fig.colorbar(pcm, ax=ax, label="Speed (m/s)", shrink=0.7)

skip = 5
u_plot = u_east.where(interior)
v_plot = v_north.where(interior)
ax.quiver(
    ds_t.nav_lon.values[::skip, ::skip], ds_t.nav_lat.values[::skip, ::skip],
    u_plot.values[::skip, ::skip], v_plot.values[::skip, ::skip],
    color="k", scale=1.5, width=0.004, transform=ccrs.PlateCarree(),
)
ax.set_title("Mean Surface Currents")
fig.tight_layout()
fig.savefig("../figures/surface_currents.png", dpi=150, bbox_inches="tight")

## Surface kinetic energy

Time-mean surface KE = ½(u² + v²). Highlights the energetic
western boundary current and inter-gyre jet.

In [ ]:
ke = (0.5 * (u_east**2 + v_north**2)).where(interior)

fig, ax = plt.subplots(figsize=(8, 7), subplot_kw=dict(projection=proj))
ax.set_extent(extent, crs=ccrs.PlateCarree())
ax.add_feature(cfeature.LAND, facecolor="tan", edgecolor="k", linewidth=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
ax.gridlines(draw_labels=False, alpha=0.3)

pcm = ax.pcolormesh(
    ds_t.nav_lon.values, ds_t.nav_lat.values, ke.values,
    shading="auto", cmap="magma",
    norm=mcolors.LogNorm(vmin=1e-5, vmax=float(ke.max())),
    transform=ccrs.PlateCarree(),
)
fig.colorbar(pcm, ax=ax, label="KE (m²/s²)", shrink=0.7)
ax.set_title("Surface Kinetic Energy")
fig.tight_layout()
fig.savefig("../figures/surface_ke.png", dpi=150, bbox_inches="tight")